In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
cd ../src/

/tf/notebooks/src


## Intialization

In [64]:
try:
    import pandas
except:
    !pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1 image-classifiers==0.2.2

### Imports

In [65]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from collections import Counter

In [66]:
from utils.metric import lwlrap

In [67]:
from params import *
from train import get_model
from params import TEST_MELS_PATH, TRAIN_MELS_PATH

In [149]:
CP_FOLDERS = [
#     "../data/densenet121_pretrained/",  # ref
#     LOG_PATH + "2021-02-04/9/",  # reproduced ref ?
#     LOG_PATH + "2021-02-04/12/",  # loader modif
#     LOG_PATH + "2021-02-04/15/",  # mixup
#    LOG_PATH + "2021-02-05/0/",  # cutmix + mixup
#     LOG_PATH + "2021-02-06/3/",  # cutmix + mixup + newfolds
    LOG_PATH + "2021-02-06/14/",
    LOG_PATH + "2021-02-06/15/",
    LOG_PATH + "2021-02-06/16/",
]

## Test

In [150]:
sub = pd.read_csv(DATA_PATH + "sample_submission.csv")

In [151]:
ref = pd.read_csv(DATA_PATH + "sub930_top5.csv")

In [152]:
preds = []

for cp in CP_FOLDERS:
    for fold_idx in range(5):
        
#         if fold_idx == 0:
        try:
            pred = np.load(cp + f"preds/pred_agg_{fold_idx}.npy")
            preds.append(pred)
        except:
            pass
    
preds = np.mean(preds, 0)

In [153]:
score = lwlrap(ref[ref.columns[1:]].values, preds)[-1]
print(f'Ref score : {score:.4f}')

Ref score : 0.9274


In [154]:
sub[sub.columns[1:]] = preds

In [155]:
sub.to_csv(OUT_DIR + F"submission_{score:.4f}.csv", index=False)

print(f'Saved submission to {OUT_DIR + F"submission_{score:.4f}.csv"}')

Saved submission to ../output/submission_0.9274.csv


In [19]:
sub.head()

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,0.176995,0.085722,0.093675,0.615611,0.072445,0.109477,0.114053,0.067227,0.084789,...,0.762768,0.118629,0.091624,0.143157,0.799194,0.085438,0.124807,0.075319,0.074638,0.138559
1,003bc2cb2,0.146267,0.169390,0.082023,0.490892,0.098180,0.067100,0.073889,0.135797,0.062457,...,0.786136,0.096164,0.973185,0.114876,0.189146,0.082111,0.080066,0.102713,0.075662,0.087977
2,0061c037e,0.356859,0.156443,0.114586,0.351453,0.134957,0.195421,0.149190,0.413765,0.152761,...,0.322519,0.178741,0.125381,0.177807,0.272414,0.122083,0.239281,0.263221,0.159463,0.198991
3,010eb14d3,0.998361,0.044152,0.049118,0.064688,0.036994,0.044312,0.045308,0.023680,0.324396,...,0.006991,0.053387,0.036333,0.038378,0.423421,0.042605,0.046380,0.110490,0.036715,0.043313
4,011318064,0.270275,0.108486,0.034820,0.250074,0.081174,0.052010,0.068074,0.072645,0.045556,...,0.999092,0.667760,0.040423,0.050682,0.570265,0.048576,0.096333,0.093938,0.046683,0.054746


## Train